In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
from tqdm import tqdm

In [2]:
if(os.path.split(os.getcwd())[1] == "misc"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = True

Current Working Directory: progettoVIPM


In [3]:
from typing import Literal

_TYPES = Literal["highest", "lowest"]

class Confusion_Matrix_stats():
    def __init__(self, filename):
        
        # Load the data from file

        self.filename = filename

        if ".npy" in filename:
            print("File is readable by numpy")
            self.cm = np.load(filename).astype("float32")
        if ".csv" in filename:
            print("File is readable by csv")
            self.cm = np.loadtxt(filename, delimiter=",", dtype=np.int64)
        
        
        self.classes = np.arange(self.cm.shape[0])

        # Compute basic metrics

        self.accuracy = self.cm.diagonal()/self.cm.sum(axis=1)
        
        # the following matrix is used to compute both the false negative and false positives by using dot prodct
        # the values will be retrieved from the diagonal of the obtained matrix
        prod_matrix = np.ones(self.cm.shape, dtype=np.int8)
        np.fill_diagonal(prod_matrix, 0)

        self.false_negative = np.dot(self.cm, prod_matrix).diagonal()

        self.false_positive = np.dot(np.transpose(self.cm), prod_matrix).diagonal()

        self.true_positive = self.cm.diagonal()

        self.real_positive = np.sum(self.cm, axis=1)

        self.real_negative = np.sum(self.cm, axis=0)

        # the true negative values are computed by excluding the rest of the values from the total sum of the matrix

        self.true_negative = -self.real_negative + self.cm.sum() - self.real_positive + self.true_positive

        self.predicted_positive = self.true_positive + self.false_positive

        self.predicted_negative = self.false_negative + self.true_negative

        # Compute advanced metrics

        self.precision = self.true_positive / self.predicted_positive

        self.recall = self.true_positive/self.real_positive

        self.f1 = 2*(self.precision * self.recall)/(self.precision + self.recall)

    def show_top_k_confused_classes(self, k):
        # This function returns the top k confused classes as a list of double values
        # the returned list is bicriterional in the sense that it returns the top k confused classes in both ways.
        # As an example: if the couple (1,2) is returned it means that the total amount of cases in which 1 was confused with 2
        # and vice versa was great

        
        # The following matrix "summed_mat" is initialized, it will eventually contain
        # the sums of the rows and columns of the confusion mat of each class
        # This is basically accomplished by summing the upper triangular part of the matrix with the lower triangular part
        # the height of the matrix is set as self.cm.shape[0]-1 because we do self.cm.shape[0]-1 iterations
        # since at every iteration the remaining parts of the matrix to confront shrinks when the last row
        # is reached there will be no confront to make
        summed_mat = np.zeros((self.cm.shape[0]-1, self.cm.shape[1]), dtype=np.int64)
        for i in range(self.cm.shape[0]-1):
            # sum the errors in the row and column relative to the current class i
            i_row = self.cm[i, i+1:]
            i_column = self.cm[i+1:, i]
            i_row = i_row + i_column
            # there is a padding of -1s in order to obtain a vector of correct lenght.
            # -1 are chosen because when doing argmax operation they will never be chosen, since in self.cm the minimum value is 0
            i_row = np.pad(i_row, (i+1,0), 'constant', constant_values=(-1))
            summed_mat[i] = i_row
        top_k= []
        print("These are the top k confused couples of classes")
        for i in range(k):
            # unravel_index is needed because np.argmax returns the position of the maximum value in the flattened array
            pos = np.unravel_index(np.argmax(summed_mat, axis=None), summed_mat.shape)
            print("#{}: {}".format(i+1, pos))
            top_k.append(pos)
            # after printing ad appending the couple we set the value at its position as -1 in order to not choose it again
            summed_mat[pos]=-1
        return top_k



    def show_metrics(self, index):
        print("---------------------------------------------------------")
        print("Showing metrics of class: {}; filename:{}".format(self.classes[index], self.filename))

        print("-------------------basic metrics------------------------")

        print("Accuracy: {}".format(self.accuracy[index]))

        print("N° of real positives: {}".format(self.real_positive[index]))
        print("N° of true positives: {}".format(self.true_positive[index]))
        print("N° of false negatives: {}".format(self.false_negative[index]))

        print("N° of real negatives: {}".format(self.real_negative[index]))
        print("N° of true negatives: {}".format(self.true_negative[index]))
        print("N° of false positives: {}".format(self.false_positive[index]))
        
        print("N° of predicted positives: {}".format(self.predicted_positive[index]))
        print("N° of predicted negatives: {}".format(self.predicted_negative[index]))

        print("-------------------advanced metrics----------------------")

        print("Precision: {}".format(self.precision[index]))
        print("Recall: {}".format(self.recall[index]))
        print("F1-score: {}".format(self.f1[index]))

        print("---------------------------------------------------------")
    
    def show_metrics_k_acc(self, k, criterion: _TYPES ="highest"):
        index_best = np.argsort(self.accuracy)[:k]
        if criterion == "lowest":
            index_best = np.argsort(self.accuracy)[-k:]
        print("Showing metrics of {} {} accuracy classes".format(criterion, k))
        for index in index_best:
            self.show_metrics(index)
    
    def show_metrics_k_tp(self, k, criterion: _TYPES ="highest"):
        index_best = np.argsort(self.true_positive)[:k]
        if criterion == "lowest":
            index_best = np.argsort(self.true_positive)[-k:]
        print("Showing metrics of {} {} true positives classes".format(criterion, k))
        for index in index_best:
            self.show_metrics(index)
    
    def show_metrics_k_tn(self, k, criterion: _TYPES ="highest"):
        index_best = np.argsort(self.true_negative)[:k]
        if criterion == "lowest":
            index_best = np.argsort(self.true_negative)[-k:]
        print("Showing metrics of {} {} true negatives classes".format(criterion, k))
        for index in index_best:
            self.show_metrics(index)
    
    def show_metrics_k_precision(self, k, criterion: _TYPES ="highest"):
        index_best = np.argsort(self.precision)[:k]
        if criterion == "lowest":
            index_best = np.argsort(self.precision)[-k:]
        print("Showing metrics of {} {} precision classes".format(criterion, k))
        for index in index_best:
            self.show_metrics(index)
    
    def show_metrics_k_recall(self, k, criterion: _TYPES ="highest"):
        index_best = np.argsort(self.recall)[:k]
        if criterion == "lowest":
            index_best = np.argsort(self.recall)[-k:]
        print("Showing metrics of {} {} recall classes".format(criterion, k))
        for index in index_best:
            self.show_metrics(index)
    
    def show_metrics_k_f1(self, k, criterion: _TYPES ="highest"):
        index_best = np.argsort(self.f1)[:k]
        if criterion == "lowest":
            index_best = np.argsort(self.f1)[-k:]
        print("Showing metrics of {} {} f1-score classes".format(criterion, k))
        for index in index_best:
            self.show_metrics(index)

In [4]:
filename = "prova.csv"
confusion_matrix = Confusion_Matrix_stats(filename)
confusion_matrix.show_metrics(0)


File is readable by csv
---------------------------------------------------------
Showing metrics of class: 0; filename:prova.csv
-------------------basic metrics------------------------
Accuracy: 0.375
N° of real positives: 16
N° of true positives: 6
N° of false negatives: 10
N° of real negatives: 17
N° of true negatives: 40
N° of false positives: 11
N° of predicted positives: 17
N° of predicted negatives: 50
-------------------advanced metrics----------------------
Precision: 0.35294117647058826
Recall: 0.375
F1-score: 0.3636363636363636
---------------------------------------------------------


In [51]:
file = np.loadtxt("train_small.csv", delimiter=",", dtype="str")
file[:,1] = file[:,1].astype("int")
unique_classes = np.unique(file[:,1], return_index=True)[0].astype("int")
unique_indexes = np.unique(file[:,1], return_index=True)[1]
holder = np.stack((unique_indexes, unique_classes), axis=-1)
# print(data_to_file)
holder = np.sort(holder, axis=0)
unique_filenames = file[holder[:,0]]
# data_to_file = np.stack((unique_filenames, holder[1,:]), axis=-1)
# print(data_to_file)

print(unique_filenames)
rows = ["{},{}".format(i, j) for i, j in unique_filenames] 
text = "\n".join(rows) 


with open('unique_classes.csv', 'w') as f: 
    f.write(text)

[['train_059371.jpg' '0']
 ['train_089398.jpg' '1']
 ['train_062381.jpg' '2']
 ['train_067031.jpg' '3']
 ['train_040012.jpg' '4']
 ['train_074191.jpg' '5']
 ['train_069104.jpg' '6']
 ['train_024255.jpg' '7']
 ['train_085321.jpg' '8']
 ['train_008697.jpg' '9']
 ['train_006793.jpg' '10']
 ['train_075674.jpg' '11']
 ['train_032521.jpg' '12']
 ['train_026411.jpg' '13']
 ['train_006384.jpg' '14']
 ['train_069559.jpg' '15']
 ['train_017653.jpg' '16']
 ['train_022920.jpg' '17']
 ['train_054633.jpg' '18']
 ['train_089890.jpg' '19']
 ['train_084813.jpg' '20']
 ['train_097940.jpg' '21']
 ['train_027769.jpg' '22']
 ['train_071221.jpg' '23']
 ['train_023901.jpg' '24']
 ['train_026767.jpg' '25']
 ['train_033664.jpg' '26']
 ['train_086742.jpg' '27']
 ['train_055797.jpg' '28']
 ['train_097400.jpg' '29']
 ['train_022421.jpg' '30']
 ['train_011628.jpg' '31']
 ['train_094889.jpg' '32']
 ['train_031523.jpg' '33']
 ['train_098387.jpg' '34']
 ['train_095781.jpg' '35']
 ['train_016178.jpg' '36']
 ['train_06

In [30]:
classes = np.arange(251)

rows = ["{}".format(i) for i in classes] 
text = "\n".join(rows) 
  
with open('disambiguation.csv', 'w') as f: 
    f.write(text)

In [56]:
import shutil
from tqdm import tqdm

file = np.loadtxt("train_small.csv", delimiter=",", dtype="str")
file[:,1] = file[:,1].astype("int")
print(file)
path = "./separated_training_set/"
for i in range(0,251):
    path_folder = path + str(i) 
    if not os.path.exists(path_folder):
        os.makedirs(path_folder)


for instance in tqdm(file):
    print(instance[0])
    path_folder = path + instance[1]
    shutil.copy("./train_set/{}".format(instance[0]), path_folder)


[['train_059371.jpg' '0']
 ['train_059389.jpg' '0']
 ['train_059425.jpg' '0']
 ...
 ['train_120161.jpg' '250']
 ['train_120200.jpg' '250']
 ['train_120211.jpg' '250']]


  3%|▎         | 167/5020 [00:00<00:02, 1659.21it/s]

train_059371.jpg
train_059389.jpg
train_059425.jpg
train_059445.jpg
train_059516.jpg
train_059538.jpg
train_059541.jpg
train_059570.jpg
train_059713.jpg
train_059719.jpg
train_059744.jpg
train_059789.jpg
train_059795.jpg
train_059837.jpg
train_059855.jpg
train_059893.jpg
train_059917.jpg
train_059919.jpg
train_059921.jpg
train_059928.jpg
train_089398.jpg
train_089407.jpg
train_089435.jpg
train_089456.jpg
train_089510.jpg
train_089516.jpg
train_089530.jpg
train_089579.jpg
train_089594.jpg
train_089613.jpg
train_089629.jpg
train_089681.jpg
train_089700.jpg
train_089720.jpg
train_089739.jpg
train_089740.jpg
train_089773.jpg
train_089783.jpg
train_089842.jpg
train_089844.jpg
train_062381.jpg
train_062412.jpg
train_062446.jpg
train_062491.jpg
train_062493.jpg
train_062510.jpg
train_062538.jpg
train_062569.jpg
train_062571.jpg
train_062577.jpg
train_062627.jpg
train_062644.jpg
train_062652.jpg
train_062665.jpg
train_062667.jpg
train_062682.jpg
train_062692.jpg
train_062702.jpg
train_062726.j

  8%|▊         | 378/5020 [00:00<00:02, 1918.03it/s]

train_006544.jpg
train_006546.jpg
train_006553.jpg
train_006557.jpg
train_006568.jpg
train_006610.jpg
train_006622.jpg
train_006641.jpg
train_006675.jpg
train_006720.jpg
train_006734.jpg
train_006751.jpg
train_006774.jpg
train_069559.jpg
train_069562.jpg
train_069579.jpg
train_069610.jpg
train_069614.jpg
train_069633.jpg
train_069651.jpg
train_069654.jpg
train_069671.jpg
train_069690.jpg
train_069691.jpg
train_069704.jpg
train_069714.jpg
train_069721.jpg
train_069731.jpg
train_069776.jpg
train_069799.jpg
train_069812.jpg
train_069821.jpg
train_069847.jpg
train_017653.jpg
train_017698.jpg
train_017713.jpg
train_017719.jpg
train_017776.jpg
train_017828.jpg
train_017846.jpg
train_017847.jpg
train_017862.jpg
train_017941.jpg
train_017959.jpg
train_017994.jpg
train_018005.jpg
train_018024.jpg
train_018025.jpg
train_018049.jpg
train_018071.jpg
train_018111.jpg
train_018126.jpg
train_018130.jpg
train_022920.jpg
train_022928.jpg
train_022941.jpg
train_022949.jpg
train_022956.jpg
train_022972.j

...

train_043393.jpg
train_043408.jpg
train_043438.jpg
train_043441.jpg
train_043455.jpg
train_043478.jpg
train_043512.jpg
train_043543.jpg
train_043564.jpg
train_043565.jpg
train_043569.jpg
train_043579.jpg
train_043588.jpg
train_043596.jpg
train_064115.jpg
train_064149.jpg
train_064195.jpg
train_064218.jpg
train_064235.jpg
train_064241.jpg
train_064253.jpg
train_064264.jpg
train_064321.jpg
train_064331.jpg
train_064339.jpg
train_064341.jpg
train_064361.jpg
train_064406.jpg
train_064411.jpg
train_064418.jpg
train_064432.jpg
train_064478.jpg
train_064518.jpg
train_064567.jpg
train_018164.jpg
train_018230.jpg
train_018244.jpg
train_018250.jpg
train_018264.jpg
train_018287.jpg
train_018295.jpg
train_018312.jpg
train_018316.jpg
train_018325.jpg
train_018360.jpg
train_018361.jpg
train_018394.jpg
train_018397.jpg
train_018405.jpg
train_018430.jpg
train_018442.jpg
train_018459.jpg
train_018477.jpg
train_018508.jpg
train_034902.jpg
train_034943.jpg
train_034961.jpg
train_034976.jpg
train_035011.j

 90%|█████████ | 4539/5020 [00:02<00:00, 2003.15it/s]

train_103091.jpg
train_103146.jpg
train_103154.jpg
train_103184.jpg
train_103188.jpg
train_103234.jpg
train_103266.jpg
train_103289.jpg
train_103324.jpg
train_103339.jpg
train_103356.jpg
train_103429.jpg
train_103452.jpg
train_103459.jpg
train_103478.jpg
train_103488.jpg
train_103491.jpg
train_103528.jpg
train_103529.jpg
train_103544.jpg
train_103556.jpg
train_103575.jpg
train_103646.jpg
train_103649.jpg
train_103656.jpg
train_103661.jpg
train_103700.jpg
train_103718.jpg
train_103749.jpg
train_103771.jpg
train_103800.jpg
train_103836.jpg
train_103841.jpg
train_103859.jpg
train_103902.jpg
train_103914.jpg
train_103921.jpg
train_103954.jpg
train_103975.jpg
train_104010.jpg
train_104189.jpg
train_104203.jpg
train_104222.jpg
train_104232.jpg
train_104237.jpg
train_104246.jpg
train_104269.jpg
train_104281.jpg
train_104296.jpg
train_104312.jpg
train_104336.jpg
train_104362.jpg
train_104390.jpg
train_104407.jpg
train_104410.jpg
train_104416.jpg
train_104445.jpg
train_104457.jpg
train_104472.j

100%|██████████| 5020/5020 [00:02<00:00, 2031.60it/s]

train_112252.jpg
train_112279.jpg
train_112323.jpg
train_112336.jpg
train_112356.jpg
train_112384.jpg
train_112414.jpg
train_112421.jpg
train_112434.jpg
train_112448.jpg
train_112471.jpg
train_112488.jpg
train_112503.jpg
train_112509.jpg
train_112511.jpg
train_112515.jpg
train_112520.jpg
train_112538.jpg
train_112550.jpg
train_112567.jpg
train_112574.jpg
train_112585.jpg
train_112600.jpg
train_112638.jpg
train_112675.jpg
train_112698.jpg
train_112708.jpg
train_112727.jpg
train_112759.jpg
train_112770.jpg
train_112775.jpg
train_112786.jpg
train_112807.jpg
train_112847.jpg
train_112856.jpg
train_112897.jpg
train_112898.jpg
train_112908.jpg
train_112961.jpg
train_112991.jpg
train_113004.jpg
train_113023.jpg
train_113037.jpg
train_113069.jpg
train_113115.jpg
train_113124.jpg
train_113176.jpg
train_113200.jpg
train_113216.jpg
train_113229.jpg
train_113239.jpg
train_113241.jpg
train_113301.jpg
train_113308.jpg
train_113350.jpg
train_113362.jpg
train_113370.jpg
train_113381.jpg
train_113429.j